In [ ]:
import os
import autogen

config_list = [
    {
        "model": "Qwen2-7B-Instruct",
        "base_url":"http://0.0.0.0:8000/v1",
        "api_key": "qwen",
    }
]

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}


### step1 config embedding function


错误示范:
```python
from chromadb.utils import embedding_functions
url="http://0.0.0.0:8200/v1"
bge_embedding_function = embedding_functions.HuggingFaceEmbeddingServer(url)
```

这个是接 `TGI` 启动的API, 但我这里使用 `FastChat` 启动的API

In [2]:
from chromadb.utils import embedding_functions

# 使用 text2vec吧，直接下载模型即可（自定义的比如bge，后面再看看）
bge_embedding_function = embedding_functions.Text2VecEmbeddingFunction()

/home/yhchen/anaconda3/envs/autogen_env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len,
    separators=["\n\n", "\n", "\r", "\t"]
)

In [ ]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent


llm_config = {
    "config_list": config_list,
    "temperature": 0,
}


assistant = RetrieveAssistantAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="""Reply step-by-step.
    You are an AI assistant.
    You have access to the following tools:""",
    human_input_mode="NEVER",
)

rag_agent = RetrieveUserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker":False},
    retrieve_config={
        'task': "qa",
        "docs_path": "./rag.pdf",
        "collection_name": "rag_collection",
        "embedding_function": bge_embedding_function,
        "custom_text_split_function": text_spliter.split_text,
    }
)

In [ ]:
assistant.reset()
rag_agent.initiate_chat(assistant, problem="What is the workflow in docGPT?", n_results=2)